# CLASSES & OOP

## Random Notes
* Classes are a reference type, as opposed to a value type (e.g. int, char, bool, etc.)
* C# can utilize memory either by using the stack or the heap, classes are stored on the heap
* When classes are no longer in use, they get automatically get de-referenced in the heap, or garbage collected
    * automated garbage collection via .NET
    * explicit garbage collection is available
* Classes inherit from the base `object`
* By default, classes have an `internal` access scope
* By convention for general C# project structure, there is one class per file and that file shares the same name as the class
* Classes can consist of __*members*__ (e.g. fields, methods, properties, events, etc.)
    * List of members:
        * *Constants*: Constant values associated with the class
        * *Fields*: Variables that are associated with the class
        * *Methods*: Actions that can be performed by the class
        * *Properties*: Actions associated with reading and writing named properties of the class
        * *Indexers*: Actions associated with indexing instances of the class like an array
        * *Events*: Notifications that can be generated by the class
        * *Operators*: Conversions and expression operators supported by the class
        * *Constructors*: Actions required to initialize instances of the class or the class itself
        * *Finalizers*: Actions done before instances of the class are permanently discarded
        * *Types*: Nested types declared by the class

<hr>

## Base Class

Additional topics include:
* __OOP: Encapsultation__ (public & private acceses)
    * for more info on access keywords, see [here](https://learn.microsoft.com/en-us/dotnet/csharp/tour-of-csharp/program-building-blocks#accessibility)
* __OOP: Abstraction__
    * NOTE: BankAccount class is seen as a base class (*similar* to an abstract class) that will be inherited by more specific bank account classes below (e.g. InterestEarningAccount, LineOfCreditAccount, GiftCardAccount)
* Getter & setter properties
* Constructors & constructor chaining/overloading
* Nullable reference types ('?')
* Exception handling & raising
* `virtual` methods (and brief on `abstract` methods)
* StringBuilder objects (kinda like Python's 3-quoted strings or .join())

In [103]:
public class Transaction {
    public decimal Amount { get; }
    public DateTime Date { get; }
    public string Description { get; }

    public Transaction(decimal amount, DateTime date, string description) {
        this.Amount = amount;
        this.Date = date;
        this.Description = description;
    }
}



// CENTRAL CLASS
public class BankAccount {
    // Shorthand getter & setter properties (automatic properties), see https://codeeasy.io/lesson/properties
    public string ID { get; }
    public string Owner { get; set; }
    public decimal Balance { 
        get { // custom getter
            decimal balance = 0;

            foreach (var transaction in _allTransactions) {
                balance += transaction.Amount;
            }

            return balance;
        } 
    }

    private static int accountNumberSeed = 101;

    private readonly decimal _minimumBalance;
    
    private List<Transaction> _allTransactions = new List<Transaction>();

    // Constructor (& constructor chaining/overloading [constructor choice depends on derived class])
    public BankAccount(string name, decimal initialBalance) : this(name, initialBalance, 0) { }

    public BankAccount(string name, decimal initialBalance, decimal minimumBalance)
    {
        this.Owner = name;
        // no need to assign 'this.Balance' in constructor since a custom getter has been written above
        // this.Balance = initialBalance; 
        this.ID = accountNumberSeed.ToString();
        accountNumberSeed++;

        _minimumBalance = minimumBalance;
        if (initialBalance > 0)
            MakeDeposit(initialBalance, DateTime.Now, "Initial balance");
    }

    public void MakeDeposit(decimal amount, DateTime date, string description) {
        if (amount <= 0) {
            // raising/throwing exceptions
            throw new ArgumentOutOfRangeException(nameof(amount), "Amount of deposit must be positive");
        }
        
        var deposit = new Transaction(amount, date, description);
        _allTransactions.Add(deposit);
    }

    /* OLD VERSION***********************************************************************
    public void MakeWithdrawal(decimal amount, DateTime date, string description) {
        if (amount <= 0) {
            throw new ArgumentOutOfRangeException(nameof(amount), "Amount of withdrawal must be positive");
        }
        if (Balance - amount < _minimumBalance) {
            throw new InvalidOperationException("Not sufficient funds for this withdrawal");
        }
        var withdrawal = new Transaction(-amount, date, description);
        allTransactions.Add(withdrawal);
    }
    */
    // NEW VERSION: nullable reference types ('?') for overdraft transactions
    // '?' annotation allows method to return null
    public void MakeWithdrawal(decimal amount, DateTime date, string note)
    {
        if (amount <= 0) {
            throw new ArgumentOutOfRangeException(nameof(amount), "Amount of withdrawal must be positive");
        }

        Transaction? overdraftTransaction = CheckWithdrawalLimit(Balance - amount < _minimumBalance);
        Transaction? withdrawal = new(-amount, date, note);

        _allTransactions.Add(withdrawal);

        if (overdraftTransaction != null)
            _allTransactions.Add(overdraftTransaction);
    }

    // `protected` access: can only be accessed from base/inherited classes (e.g. cannot be directly called from main())
    // `virtual` keyword gives inherited classes the (optional) ability to override base class implementation --> LineOfCreditAccount class
    protected virtual Transaction? CheckWithdrawalLimit(bool isOverdrawn)
    {
        if (isOverdrawn) {
            throw new InvalidOperationException("Not sufficient funds for this withdrawal");
        }
        else {
            return default;
        }
    }

    // Will be used to demonstrate polymorphism in InterestEarningAccount class below
    // `virtual` keyword gives inherited classes the (optional) ability to override base class implementation
    // an `abstract` keyword is similar to `virtual`, but it is required to override base class implementation
    public virtual void PerformMonthEndTransactions() { }


    // Using a StringBuilder object to get transaction history
    // StringBuilders are kinda like Python's 3-quoted strings or .join()
    public string GetTransactionStatement() {
        var report = new StringBuilder(); // in System.Text 

        //HEADER
        report.AppendLine("Date\t\tAmount\tDescription"); // '\t' is tabbed whitespace

        foreach (var transaction in _allTransactions) {
            //ROWS
            // DateTime.ToShortDateString() excludes the time and only includes the date
            report.AppendLine($"{transaction.Date.ToShortDateString()}\t${transaction.Amount}\t{transaction.Description}");
        }

        return report.ToString();
    }
}


### *just pretend that the cells below are the Main method for now*

In [83]:
var account1 = new BankAccount("Squidward", 260);
Console.WriteLine($"New bank account for {account1.Owner} [Account ID #{account1.ID}], starting with ${account1.Balance}");

New bank account for Squidward [Account ID #101], starting with $260


*NOTE: Parameter-less class constructors can also this syntax to create new instances:*

*ANOTHER NOTE: For classes that would only consist of properties (like below), structs might be a better alternative for performance*

In [116]:
class BankAccountParamLess {
    public string ID { get; }
    public string Owner { get; set; }
    public decimal Balance { get; set; }

    private static int _accountNumberSeed = 201;

    public BankAccountParamLess() {
        this.ID = _accountNumberSeed.ToString();
        _accountNumberSeed++;
    }
}



// MAIN
var account2 = new BankAccountParamLess(){
    Owner = "Spongebob",
    Balance = 0.21m
};

Console.WriteLine($"New bank account for {account2.Owner} [Account ID #{account2.ID}], starting with ${account2.Balance}");

New bank account for Spongebob [Account ID #201], starting with $0.21


In [84]:
account1.MakeWithdrawal(75, DateTime.Now, "Clarinet");
Console.WriteLine($"$75 has been debited from {account1.Owner}'s account. Remaining balance: ${account1.Balance}");

$75 has been debited from Squidward's account. Remaining balance: $185


Handling exceptions (try/catch):

In [85]:
try{
    account1.MakeDeposit(-300, DateTime.Now, "tax fraud");
}
catch (ArgumentOutOfRangeException e){
    Console.WriteLine("Attempting to deposit a negative amount, cancelling transaction...");
    Console.WriteLine($"Exception is as follows: {e.ToString()}");
}

Attempting to deposit a negative amount, cancelling transaction...
Exception is as follows: System.ArgumentOutOfRangeException: Amount of deposit must be positive (Parameter 'amount')
   at Submission#83.BankAccount.MakeDeposit(Decimal amount, DateTime date, String description)
   at Submission#86.<<Initialize>>d__0.MoveNext()


In [86]:
try{
    account1.MakeWithdrawal(9175, DateTime.Now, "Rent");
    Console.WriteLine($"$9175 has been debited from {account1.Owner}'s account. Remaining balance: ${account1.Balance}");
}
catch (InvalidOperationException e){
    Console.WriteLine("You're broke INSUFFICIENT FUNDS");
    Console.WriteLine($"Exception is as follows: {e.ToString()}");
}

You're broke INSUFFICIENT FUNDS
Exception is as follows: System.InvalidOperationException: Not sufficient funds for this withdrawal
   at Submission#83.BankAccount.MakeWithdrawal(Decimal amount, DateTime date, String description)
   at Submission#87.<<Initialize>>d__0.MoveNext()


Using method that uses StringBuilder object to get print out of all transaction history:

In [87]:
Console.WriteLine(account1.GetTransactionStatement());

Date		Amount	Description
5/3/2023	$260	Initial balance
5/3/2023	$-75	Clarinet



<hr>

## Inheritance, Polymorphism, & Overrides

Additional topics include:
* Inherited/derived class constructors
* Optional/default parameters

In [108]:
public class InterestEarningAccount : BankAccount {

    // If constructor is defined in base class, derived class must also declare a constructor that includes all parameters
    // from base class constructor
    public InterestEarningAccount(string name, decimal initialBalance) : base(name, initialBalance) { }

    // derived from virtual method in BankAccount
    public override void PerformMonthEndTransactions() {
        if (Balance > 500m)
        {
            decimal interest = Balance * 0.05m;
            MakeDeposit(interest, DateTime.Now, "apply monthly interest");
        }
    }
}




public class LineOfCreditAccount : BankAccount {
    // using contructor chaining from base class
    public LineOfCreditAccount(string name, decimal initialBalance, decimal creditLimit) : base(name, initialBalance, -creditLimit) { }

    protected override Transaction? CheckWithdrawalLimit(bool isOverdrawn) =>
        isOverdrawn
        ? new Transaction(-2000, DateTime.Now, "Oops small overdraft fee <3")
        : default;

    // derived from virtual method in BankAccount
    public override void PerformMonthEndTransactions()
    {
        if (Balance < 0)
        {
            // Negate the balance to get a positive interest charge:
            decimal interest = -Balance * 0.07m;
            MakeWithdrawal(interest, DateTime.Now, "Charge monthly interest");
        }
    }
}




public class GiftCardAccount : BankAccount {
    private readonly decimal _monthlyDeposit = 0m;

    // optional/default parameter: monthlyDeposit
    public GiftCardAccount(string name, decimal initialBalance, decimal monthlyDeposit = 0) : base(name, initialBalance)
        => _monthlyDeposit = monthlyDeposit;

    public override void PerformMonthEndTransactions()
    {
        if (_monthlyDeposit != 0)
        {
            MakeDeposit(_monthlyDeposit, DateTime.Now, "Add monthly deposit");
        }
    }
    
}

In [89]:
var giftCard = new GiftCardAccount("gift card", 100, 50);
giftCard.MakeWithdrawal(20, DateTime.Now, "get expensive coffee");
giftCard.MakeWithdrawal(50, DateTime.Now, "buy groceries");
giftCard.PerformMonthEndTransactions();
// can make additional deposits:
giftCard.MakeDeposit(27.50m, DateTime.Now, "add some additional spending money");
Console.WriteLine(giftCard.GetTransactionStatement());

var savings = new InterestEarningAccount("savings account", 10000);
savings.MakeDeposit(750, DateTime.Now, "save some money");
savings.MakeDeposit(1250, DateTime.Now, "Add more savings");
savings.MakeWithdrawal(250, DateTime.Now, "Needed to pay monthly bills");
savings.PerformMonthEndTransactions();
Console.WriteLine(savings.GetTransactionStatement());

Date		Amount	Description
5/3/2023	$100	Initial balance
5/3/2023	$-20	get expensive coffee
5/3/2023	$-50	buy groceries
5/3/2023	$50	Add monthly deposit
5/3/2023	$27.50	add some additional spending money

Date		Amount	Description
5/3/2023	$10000	Initial balance
5/3/2023	$750	save some money
5/3/2023	$1250	Add more savings
5/3/2023	$-250	Needed to pay monthly bills
5/3/2023	$587.50	apply monthly interest



Sufficient line of credit:

In [97]:
var lineOfCredit = new LineOfCreditAccount("line of credit", 0, 11000);

// How much is too much to borrow?
lineOfCredit.MakeWithdrawal(1000m, DateTime.Now, "Take out monthly advance");
lineOfCredit.MakeDeposit(50m, DateTime.Now, "Pay back small amount");
lineOfCredit.MakeWithdrawal(5000m, DateTime.Now, "Emergency funds for repairs");
lineOfCredit.MakeDeposit(150m, DateTime.Now, "Partial restoration on repairs");
lineOfCredit.PerformMonthEndTransactions();
Console.WriteLine(lineOfCredit.GetTransactionStatement());

Date		Amount	Description
5/3/2023	$-1000	Take out monthly advance
5/3/2023	$50	Pay back small amount
5/3/2023	$-5000	Emergency funds for repairs
5/3/2023	$150	Partial restoration on repairs
5/3/2023	$-406.00	Charge monthly interest



Insufficient line of credit:

In [98]:
var lineOfCredit = new LineOfCreditAccount("line of credit", 0, 500);

// How much is too much to borrow?
lineOfCredit.MakeWithdrawal(1000m, DateTime.Now, "Take out monthly advance");
lineOfCredit.MakeDeposit(50m, DateTime.Now, "Pay back small amount");
lineOfCredit.MakeWithdrawal(5000m, DateTime.Now, "Emergency funds for repairs");
lineOfCredit.MakeDeposit(150m, DateTime.Now, "Partial restoration on repairs");
lineOfCredit.PerformMonthEndTransactions();
Console.WriteLine(lineOfCredit.GetTransactionStatement());

Error: System.InvalidOperationException: Not sufficient funds for this withdrawal
   at Submission#83.BankAccount.MakeWithdrawal(Decimal amount, DateTime date, String description)
   at Submission#99.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Overdraft transactions:

In [109]:
var lineOfCredit = new LineOfCreditAccount("line of credit", 0, 2000);

// How much is too much to borrow?
lineOfCredit.MakeWithdrawal(1000m, DateTime.Now, "Take out monthly advance");
lineOfCredit.MakeDeposit(50m, DateTime.Now, "Pay back small amount");
lineOfCredit.MakeWithdrawal(5000m, DateTime.Now, "Emergency funds for repairs");
lineOfCredit.MakeDeposit(150m, DateTime.Now, "Partial restoration on repairs");
lineOfCredit.PerformMonthEndTransactions();
Console.WriteLine(lineOfCredit.GetTransactionStatement());

Date		Amount	Description
5/3/2023	$-1000	Take out monthly advance
5/3/2023	$50	Pay back small amount
5/3/2023	$-5000	Emergency funds for repairs
5/3/2023	$-2000	Oops small overdraft fee <3
5/3/2023	$150	Partial restoration on repairs
5/3/2023	$-546.00	Charge monthly interest
5/3/2023	$-2000	Oops small overdraft fee <3



<hr>

## RANDOM CLASS EXAMPLE:

In [4]:
class SportBase
{
    protected string SportType { get; set; }
    protected string TeamName { get; set; }
    protected string PlayerName { get; set; }

    public SportBase(string sporttype, string teamname, string playername)
    {
        this.SportType = sporttype;
        this.TeamName = teamname;
        this.PlayerName = playername;
        CreationMessage();
    }


    public void CreationMessage()
    {
        Console.WriteLine($"Creating {this.SportType} object...");
    }

    public virtual void PlayerAffiliation()
    {
        Console.WriteLine($"{this.PlayerName} plays {this.SportType} for the {this.TeamName}.");
    }
}


class Hockey : SportBase 
{
    private string NationalLeague = "National Hockey League (NHL)";

    public Hockey(string sporttype, string teamname, string playername) 
        : base(sporttype, teamname, playername) { }


    public override void PlayerAffiliation()
    {
        Console.WriteLine($"{this.PlayerName} plays in the {this.NationalLeague} for the {this.TeamName}.");
    }

}

In [5]:
Hockey h = new Hockey("Hockey", "Colorado Avalanche", "Gabriel Landeskog");

Creating Hockey object...


In [6]:
h.PlayerAffiliation();

Gabriel Landeskog plays in the National Hockey League (NHL) for the Colorado Avalanche.
